In [1]:
import os,sys,json
from glob import glob

os.chdir('/home/avijit/projects/Awareness_vs_Unawareness')
sys.path.insert(0,"/home/avijit/projects/Awareness_vs_Unawareness/robustfairnesscode")

In [2]:
import warnings
warnings.filterwarnings('ignore')

from robustfairnesscode import data, losses, optimization, model, utils, dro_training, softweights_training
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder

In [3]:
def writejson(path,arr,name):
    try:
        os.mkdir(path+'preds/')
    except:
        pass
    f = open(path+'preds/'+name,'w')
    json.dump(arr,f)
    f.close()

In [4]:
path = 'data/datasets/uci_adult/'
synthfols = glob("data/datasets/uci_adult/synthetic/*/")
paths = [path]
paths.extend(synthfols)

In [5]:
def task(path):
    
    print(path)

    if os.path.exists(path+'preds/Softweights_pred.json') == False:
    
        train_df = pd.read_csv(path+'train.csv',header=None)

        test_df = pd.read_csv(path+'test.csv',header=None)

        train_df.columns = ['age', 'workclass', 'fnlwgt','education', 'education-num', 'marital-status',
               'occupation', 'relationship', 'race', 'sex', 'capital-gain',
               'capital-loss', 'hours-per-week', 'native-country','income']

        test_df.columns = ['age', 'workclass', 'fnlwgt','education', 'education-num', 'marital-status',
               'occupation', 'relationship', 'race', 'sex', 'capital-gain',
               'capital-loss', 'hours-per-week', 'native-country','income']

        #test_df.index = test_df['sex']

        test_df = test_df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

        # train_df['sex'] = train_df['sex'].map({'Male': 1, 'Female': 0})
        # test_df['sex'] = test_df['sex'].map({'Male': 1, 'Female': 0})

        X_train = train_df.drop(['income'],axis=1)
        X_test = test_df.drop(['income'],axis=1)

        y_train = pd.Series(train_df['income'])
        y_test = pd.Series(test_df['income'])


        y_train = pd.Series(y_train.factorize(sort=True)[0], index=y_train.index)
        y_test = pd.Series(y_test.factorize(sort=True)[0], index=y_test.index)

        X_merged = pd.concat([X_train,X_test])

        ohe = make_column_transformer(
            (OneHotEncoder(sparse=False), X_merged.dtypes == 'object'),
            remainder='passthrough', verbose_feature_names_out=False)

        X_merged_temp  = pd.DataFrame(ohe.fit_transform(X_merged), columns=ohe.get_feature_names_out(), index=X_merged.index)

        train_df  = pd.DataFrame(ohe.transform(X_train), columns=ohe.get_feature_names_out(), index=X_train.index)
        test_df = pd.DataFrame(ohe.transform(X_test), columns=ohe.get_feature_names_out(), index=X_test.index)

        train_df['income'] = y_train
        test_df['income'] = y_test

        print(train_df.columns)

        LABEL_COLUMN = "income"
        FEATURE_NAMES = list(train_df.columns)
        FEATURE_NAMES.remove(LABEL_COLUMN)
        PROTECTED_COLUMNS = ['sex_Female','sex_Male']

        PROXY_COLUMNS = PROTECTED_COLUMNS 


        val = dro_training.get_results_for_learning_rates(train_df, test_df, FEATURE_NAMES, 
                                                          PROTECTED_COLUMNS, PROXY_COLUMNS, LABEL_COLUMN, num_loops = 10,constraint='tpr_and_fpr')

        vall = [float(v) for v in val[0]]

        val_m = [0 if i < 0 else 1 for i in vall]

        writejson(path,val_m,'DRO_pred.json')

        val2 = softweights_training.get_results_for_learning_rates(train_df, test_df, FEATURE_NAMES, 
                                                                   PROTECTED_COLUMNS, PROXY_COLUMNS, LABEL_COLUMN, num_loops = 10, constraint='tpr_and_fpr')

        val2l = [float(v) for v in val2[0]]

        val_m2 = [0 if i < 0 else 1 for i in val2l]

        writejson(path,val_m2,'Softweights_pred.json')

        return path, 'DRO and SW done'

In [10]:
for path in paths:
    try:
        task(path)
    except:
        pass

data/datasets/uci_adult/
data/datasets/uci_adult/synthetic/gender_flip_labels0.8_version8/
data/datasets/uci_adult/synthetic/gender_flip_labels0.8_version9/
data/datasets/uci_adult/synthetic/gender_flip_labels0.9_version6/
data/datasets/uci_adult/synthetic/gender_flip_labels0.9_version7/
data/datasets/uci_adult/synthetic/gender_flip_labels0.9_version8/
data/datasets/uci_adult/synthetic/gender_flip_labels0.9_version9/
data/datasets/uci_adult/synthetic/gender_flip_labels1.0_version0/
data/datasets/uci_adult/synthetic/gender_flip_labels1.0_version1/
data/datasets/uci_adult/synthetic/gender_flip_labels1.0_version2/
data/datasets/uci_adult/synthetic/gender_flip_labels1.0_version3/
data/datasets/uci_adult/synthetic/gender_flip_labels1.0_version4/
data/datasets/uci_adult/synthetic/gender_flip_labels1.0_version5/
data/datasets/uci_adult/synthetic/gender_flip_labels1.0_version6/
data/datasets/uci_adult/synthetic/gender_flip_labels1.0_version7/
data/datasets/uci_adult/synthetic/gender_flip_label

In [1]:
# from multiprocessing.pool import ThreadPool as Pool

# pool = Pool(10)
# for result in pool.imap_unordered(task, paths):
#     print(result)

In [ ]:
#   Args:
#     predictions: numpy array of floats representing predictions. Predictions are treated 
#       as positive classification if value is >0, and negative classification if value is <= 0.
#     labels: numpy array of floats representing labels. labels are also treated as positive 
#       classification if value is >0, and negative classification if value is <= 0.